<a href="https://colab.research.google.com/github/nahbos/AUT-Neural-Networks/blob/main/HW01/HW01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**Copyright (C) 2022 Sobhan Moradian Daghigh**
######**Date: 3/1/2022**

In [73]:
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from sklearn import preprocessing

In [2]:
print(tf.__version__)

2.8.0


In [184]:
url = 'https://raw.githubusercontent.com/nahbos/AUT-Neural-Networks/main/HW01/Data/crx.data'
dataset = pd.read_csv(url, names=['Gender', 'Age', 'Debt', 'Married', 'BankCustomer', 'EducationLevel', 'Ethnicity', 'YearsEmployed', 
                                  'PriorDefault', 'Employed', 'CreditScore', 'DriversLicense', 'Citizen', 'ZipCode', 'Income', 'ApprovalStatus'])
dataset.head()

,Gender,Age,Debt,Married,BankCustomer,EducationLevel,Ethnicity,YearsEmployed,PriorDefault,Employed,CreditScore,DriversLicense,Citizen,ZipCode,Income,ApprovalStatus
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


In [185]:
dataset = dataset.replace('?', np.NaN)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Gender          678 non-null    object 
 1   Age             678 non-null    object 
 2   Debt            690 non-null    float64
 3   Married         684 non-null    object 
 4   BankCustomer    684 non-null    object 
 5   EducationLevel  681 non-null    object 
 6   Ethnicity       681 non-null    object 
 7   YearsEmployed   690 non-null    float64
 8   PriorDefault    690 non-null    object 
 9   Employed        690 non-null    object 
 10  CreditScore     690 non-null    int64  
 11  DriversLicense  690 non-null    object 
 12  Citizen         690 non-null    object 
 13  ZipCode         677 non-null    object 
 14  Income          690 non-null    int64  
 15  ApprovalStatus  690 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 86.4+ KB


In [197]:
useless  = ['ZipCode']
ohe_list = ['Married', 'BankCustomer', 'Citizen']

In [191]:
for x in useless:
  dataset = dataset.drop(x, axis=1)

In [192]:
nan_columns = dataset.columns[dataset.isnull().any()]

In [193]:
def fill_missing_values(dataset, nan_columns):
  for col in nan_columns:
    if dataset[col].dtypes == 'object':
      dataset[col] = dataset[col].fillna(dataset[col].value_counts().index[0])
    else:
      dataset[col] = dataset[col].fillna(dataset[col].mean(), inplace=True)
  return dataset

In [194]:
dataset = fill_missing_values(dataset, nan_columns)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Gender          690 non-null    object 
 1   Age             690 non-null    object 
 2   Debt            690 non-null    float64
 3   Married         690 non-null    object 
 4   BankCustomer    690 non-null    object 
 5   EducationLevel  690 non-null    object 
 6   Ethnicity       690 non-null    object 
 7   YearsEmployed   690 non-null    float64
 8   PriorDefault    690 non-null    object 
 9   Employed        690 non-null    object 
 10  CreditScore     690 non-null    int64  
 11  DriversLicense  690 non-null    object 
 12  Citizen         690 non-null    object 
 13  Income          690 non-null    int64  
 14  ApprovalStatus  690 non-null    object 
dtypes: float64(2), int64(2), object(11)
memory usage: 81.0+ KB


In [195]:
for col in dataset.columns:
  if dataset[col].dtypes == 'object':
    print(dataset[col].unique()[:5])

['b' 'a']
['30.83' '58.67' '24.50' '27.83' '20.17']
['u' 'y' 'l']
['g' 'p' 'gg']
['w' 'q' 'm' 'r' 'cc']
['v' 'h' 'bb' 'ff' 'j']
['t' 'f']
['t' 'f']
['f' 't']
['g' 's' 'p']
['+' '-']


In [196]:
def ohe(dataset, col):
  pd.get_dummies(dataset.loc[:, col])
  attr = pd.get_dummies(dataset[col], prefix=col + '_')
  dataset = pd.merge(
      left=dataset,
      right=attr,
      left_index=True,
      right_index=True)

  dataset.pop(col)
  return dataset

In [131]:
def label_encoder(dataset, col):
  encoder = preprocessing.LabelEncoder()
  dataset[col] = encoder.fit_transform(dataset.loc[:, col])
  return dataset

In [198]:
def encoder(dataset):
  conv_dataset = dataset.copy()
  for col in dataset.columns:
    if dataset[col].dtypes == 'object':
      try:
        conv_dataset[col] = pd.to_numeric(conv_dataset[col], downcast="float")
        print(' |_ {}: Str2Float'.format(col))

      except:
        if len(conv_dataset[col].unique()) in range(3, 10) and col in ohe_list:
          print(' |_ {}: OHE'.format(col))
          conv_dataset = ohe(conv_dataset, col)

        else:
          print(' |_ {}: LabelEncoder'.format(col))
          conv_dataset = label_encoder(conv_dataset, col)

  return conv_dataset

In [200]:
conv_dataset = encoder(dataset)
conv_dataset['ApprovalStatus'] = conv_dataset.pop('ApprovalStatus')
conv_dataset.head()

 |_ Gender: LabelEncoder
 |_ Age: Str2Float
 |_ Married: OHE
 |_ BankCustomer: OHE
 |_ EducationLevel: LabelEncoder
 |_ Ethnicity: LabelEncoder
 |_ PriorDefault: LabelEncoder
 |_ Employed: LabelEncoder
 |_ DriversLicense: LabelEncoder
 |_ Citizen: OHE
 |_ ApprovalStatus: LabelEncoder


,Gender,Age,Debt,EducationLevel,Ethnicity,YearsEmployed,PriorDefault,Employed,CreditScore,DriversLicense,Income,Married__l,Married__u,Married__y,BankCustomer__g,BankCustomer__gg,BankCustomer__p,Citizen__g,Citizen__p,Citizen__s,ApprovalStatus
0,1,30.830000,0.000,12,7,1.25,1,1,1,0,0,0,1,0,1,0,0,1,0,0,0
1,0,58.669998,4.460,10,3,3.04,1,1,6,0,560,0,1,0,1,0,0,1,0,0,0
2,0,24.500000,0.500,10,3,1.50,1,0,0,0,824,0,1,0,1,0,0,1,0,0,0
3,1,27.830000,1.540,12,7,3.75,1,1,5,1,3,0,1,0,1,0,0,1,0,0,0
4,1,20.170000,5.625,12,7,1.71,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0


In [205]:
X_temp, X_test, y_temp, y_test = train_test_split(conv_dataset.iloc[:, :-1], conv_dataset.iloc[:, -1], test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.125, random_state=42)

In [206]:
X_train.shape

(483, 20)

In [207]:
X_test.shape

(138, 20)

In [208]:
X_val.shape

(69, 20)